In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, ReLU,LSTM, BatchNormalization,Normalization
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tensorflow_model_optimization.quantization.keras import QuantizeConfig
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer
from tensorflow_model_optimization.quantization.keras import quantize_annotate_model
from tensorflow_model_optimization.quantization.keras import quantize_apply
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate
from sklearn.utils.class_weight import compute_class_weight
import pickle

# Extracting Data

In [213]:
import zipfile
import os

zip_file_path = 'zipped_folders.zip'
output_dir = 'Cuvette'
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Folders unzipped to {output_dir}")


Folders unzipped to Cuvette


# Training and Converting Model to Tflite for Cuvette 45

In [225]:
df=pd.read_csv('Cuvette/Cuvette45/data_train.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape


(93992, 10)

In [226]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate

def quantized_model():
    num_features = 10
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(128, activation='relu')(deep_input)
    hidden2 = Dense(64, activation='relu')(hidden1)
    hidden3 = Dense(32, activation='relu')(hidden1)
    hidden4 = Dense(16, activation='relu')(hidden2)
    concatenated_layers = concatenate([wide_input, hidden4])
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    
    return model
base_model = quantized_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette45',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y_train,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette45')

model.evaluate((x_test,x_test), y_test)



In [227]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [228]:
MODEL_DIR='model_cuvette45'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model45-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

2024-08-05 05:48:52.553485: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-08-05 05:48:52.553502: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-08-05 05:48:52.553599: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: model_cuvette45
2024-08-05 05:48:52.554891: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-08-05 05:48:52.554897: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: model_cuvette45
2024-08-05 05:48:52.558834: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-08-05 05:48:52.594970: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: model_cuvette45
2024-08-05 05:48:52.605768: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 52170 micr

In [229]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.int8'>
output:  <class 'numpy.int8'>


In [230]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_input_8:0', 'index': 0, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.01612352766096592, -107), 'quantization_parameters': {'scales': array([0.01612353], dtype=float32), 'zero_points': array([-107], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_7:0', 'index': 1, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.06140619516372681, -123), 'quantization_parameters': {'scales': array([0.0614062], dtype=float32), 'zero_points': array([-123], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 15, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameter

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 = 0.01612353
zero_point_input_1 = -107
scale_input_2 = 0.0614062
zero_point_input_2 = -123
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette45/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=scaler.transform(x)
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    #print(quantized_output)
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
    #print(probability)
   
    predicted_label = 1 if probability > threshold else 0  

    # Check accuracy
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Training and Converting Model to Tflite For Cuvette 47

In [231]:
df=pd.read_csv('Cuvette/Cuvette47/data_train.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape


(187176, 10)

In [232]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
def quantized_model():
    num_features = 10
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(128, activation='relu')(deep_input)
    hidden2 = Dense(64, activation='relu')(hidden1)
    hidden3 = Dense(32, activation='relu')(hidden1)
    hidden4 = Dense(16, activation='relu')(hidden2)
    concatenated_layers = concatenate([wide_input, hidden4])
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    
    return model
base_model = quantized_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette47',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y_train,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette47')

model.evaluate((x_test,x_test), y_test)


In [233]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [234]:
MODEL_DIR='model_cuvette47'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model47-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

2024-08-05 05:49:15.459341: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-08-05 05:49:15.459357: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-08-05 05:49:15.459450: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: model_cuvette47
2024-08-05 05:49:15.460725: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-08-05 05:49:15.460731: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: model_cuvette47
2024-08-05 05:49:15.464583: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-08-05 05:49:15.500865: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: model_cuvette47
2024-08-05 05:49:15.510050: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 50600 micr

In [235]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.int8'>
output:  <class 'numpy.int8'>


In [236]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_input_10:0', 'index': 0, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.016223566606640816, -106), 'quantization_parameters': {'scales': array([0.01622357], dtype=float32), 'zero_points': array([-106], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_9:0', 'index': 1, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.06672343611717224, -123), 'quantization_parameters': {'scales': array([0.06672344], dtype=float32), 'zero_points': array([-123], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 15, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parame

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 = 0.01622357
zero_point_input_1 = -106
scale_input_2 = 0.06672344
zero_point_input_2 = -123
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette47/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=scaler.transform(x)
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
   
    predicted_label = 1 if probability > threshold else 0  
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Training and Converting Model to Tflite for Cuvette 22

In [237]:
df=pd.read_csv('Cuvette/Cuvette22/data_train.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape

(93952, 10)

In [238]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
def quantized_model():
    num_features = 10
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(512, activation='relu')(deep_input)
    hidden2 = Dense(256, activation='relu')(hidden1)
    hidden3 = Dense(128, activation='relu')(hidden2)
    hidden4 = Dense(64, activation='relu')(hidden3)
    hidden5 = Dense(32, activation='relu')(hidden4)
    hidden6 = Dense(16, activation='relu')(hidden5)
    hidden7=Dense(256, activation='relu')(deep_input)
    hidden8=Dense(128, activation='relu')(deep_input)
    hidden9=Dense(64, activation='relu')(deep_input)
    hidden10=Dense(32, activation='relu')(deep_input)
    concatenated_layers = concatenate([wide_input, hidden6,hidden7,hidden8,hidden9,hidden10])
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    
    return model
base_model = quantized_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette22',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y_train,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette22')

model.evaluate((x_test,x_test), y_test)


In [239]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [240]:
MODEL_DIR='model_cuvette22'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model22-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

2024-08-05 05:50:22.615968: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-08-05 05:50:22.615989: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-08-05 05:50:22.616094: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: model_cuvette22
2024-08-05 05:50:22.618683: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-08-05 05:50:22.618691: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: model_cuvette22
2024-08-05 05:50:22.626856: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-08-05 05:50:22.704052: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: model_cuvette22
2024-08-05 05:50:22.723185: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 107091 mic

In [241]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.int8'>
output:  <class 'numpy.int8'>


In [242]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_input_16:0', 'index': 0, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.06492095440626144, -123), 'quantization_parameters': {'scales': array([0.06492095], dtype=float32), 'zero_points': array([-123], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_15:0', 'index': 1, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (3.3731067180633545, -128), 'quantization_parameters': {'scales': array([3.3731067], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 36, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_paramete

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 = 0.06492095
zero_point_input_1 = -123
scale_input_2 = 3.3731067
zero_point_input_2 = -128
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette22/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=scaler.transform(x)
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
   
    predicted_label = 1 if probability > threshold else 0  
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Training and Converting Model for Cuvette 9 

In [243]:
df=pd.read_csv('Cuvette/Cuvette9/data_train.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape

(93944, 10)

In [244]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
def quantized_model():
    num_features = 10
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(1024, activation='relu')(deep_input)
    hidden2 = Dense(512, activation='relu')(hidden1)
    hidden3 = Dense(256, activation='relu')(hidden2)
    hidden4 = Dense(128, activation='relu')(hidden3)
    hidden5 = Dense(64, activation='relu')(hidden4)
    hidden6 = Dense(32, activation='relu')(hidden5)
    hidden7 = Dense(16, activation='relu')(hidden6)
    hidden8=Dense(128, activation='relu')(wide_input)
    hidden9=Dense(64, activation='relu')(hidden8)
    hidden10=Dense(32, activation='relu')(hidden9)
    concatenated_layers = concatenate([wide_input, hidden7,hidden8,hidden9,hidden10])
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    
    return model
base_model = quantized_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette9',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y_train,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette9')

model.evaluate((x_test,x_test), y_test)


In [245]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [246]:
MODEL_DIR='model_cuvette9'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model9-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

2024-08-05 05:51:53.969899: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-08-05 05:51:53.969922: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-08-05 05:51:53.970041: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: model_cuvette9
2024-08-05 05:51:53.973058: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-08-05 05:51:53.973065: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: model_cuvette9
2024-08-05 05:51:53.982800: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-08-05 05:51:54.065792: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: model_cuvette9
2024-08-05 05:51:54.086908: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 116866 micros

In [247]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.int8'>
output:  <class 'numpy.int8'>


In [248]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_input_18:0', 'index': 0, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.027775896713137627, -116), 'quantization_parameters': {'scales': array([0.0277759], dtype=float32), 'zero_points': array([-116], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_17:0', 'index': 1, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.027775896713137627, -116), 'quantization_parameters': {'scales': array([0.0277759], dtype=float32), 'zero_points': array([-116], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 39, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parame

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 = 0.0277759
zero_point_input_1 = -116
scale_input_2 = 0.0277759
zero_point_input_2 = -116
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette9/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=scaler.transform(x)
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
   
    predicted_label = 1 if probability > threshold else 0  
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Training and Converting Model to Tflite for Cuvette 29

In [ ]:
df=pd.read_csv('Cuvette/Cuvette29/data_train.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape

In [ ]:
def quantized_model():
    num_features = 10
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(512, activation='relu')(deep_input)
    hidden2 = Dense(256, activation='relu')(hidden1)
    hidden3 = Dense(128, activation='relu')(hidden2)
    hidden4 = Dense(64, activation='relu')(hidden3)
    hidden5 = Dense(32, activation='relu')(hidden4)
    hidden6 = Dense(16, activation='relu')(hidden5)
    hidden7=Dense(512, activation='relu')(deep_input)
    hidden8=Dense(256, activation='relu')(deep_input)
    hidden9=Dense(128, activation='relu')(deep_input)
    hidden10=Dense(64, activation='relu')(deep_input)
    hidden11=Dense(32, activation='relu')(deep_input)
    concatenated_layers = concatenate([wide_input, hidden6,hidden7,hidden8,hidden9,hidden10,hidden11])
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    
    return model
base_model = quantized_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette29',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y_train,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette29')

model.evaluate((x_test,x_test), y_test)

In [ ]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [ ]:
MODEL_DIR='model_cuvette29'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model29-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 =21384.0390625
zero_point_input_1 = -128
scale_input_2 = 21384.0390625
zero_point_input_2 = -128
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette29/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x.iloc[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
   
    predicted_label = 1 if probability > threshold else 0  
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Training and Converting Model to Tflite for Cuvette37

In [249]:
df=pd.read_csv('Cuvette/Cuvette37/data_train.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
x_train.shape

(94024, 10)

In [250]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
def create_modified_model():
    num_features = 10  
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(200,activation='relu')(deep_input)
    hidden2 = Dense(190,activation='relu')(hidden1)
    hidden3 = Dense(180,activation='relu')(hidden2)
    hidden4 = Dense(170,activation='relu')(hidden3)
    hidden5 = Dense(160,activation='relu')(hidden4)
    hidden6 = Dense(150,activation='relu')(hidden5)
    hidden7 = Dense(100,activation='relu')(hidden6)
    hidden8 = Dense(80,activation='relu')(hidden7)
    hidden9 = Dense(60,activation='relu')(hidden8)
    hidden10 = Dense(32,activation='relu')(hidden9)
    parallel_input2= Dense(128,activation='relu')(deep_input)
    parallel_input3= Dense(64, activation='relu')(deep_input)
    parallel_input4= Dense(32,activation='relu')(wide_input)
    parallel_input1= Dense(256, activation='relu')(deep_input)
    parallel_input5 = Dense(512, activation='relu')(deep_input)
    concatenated_layers = concatenate([wide_input,hidden10,parallel_input1,parallel_input2,parallel_input3,parallel_input4,parallel_input5])
    
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    return model


base_model = create_modified_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()


In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette37',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y_train,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette37')

model.evaluate((x_test,x_test), y_test)

In [251]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [252]:
MODEL_DIR='model_cuvette37'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model37-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

2024-08-05 05:52:44.507095: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-08-05 05:52:44.507119: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-08-05 05:52:44.507237: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: model_cuvette37
2024-08-05 05:52:44.510889: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-08-05 05:52:44.510898: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: model_cuvette37
2024-08-05 05:52:44.522832: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-08-05 05:52:44.631250: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: model_cuvette37
2024-08-05 05:52:44.658175: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 150938 mic

In [253]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.int8'>
output:  <class 'numpy.int8'>


In [254]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_input_25:0', 'index': 0, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.502932071685791, -127), 'quantization_parameters': {'scales': array([0.5029321], dtype=float32), 'zero_points': array([-127], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'serving_default_input_26:0', 'index': 1, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.502932071685791, -127), 'quantization_parameters': {'scales': array([0.5029321], dtype=float32), 'zero_points': array([-127], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 52, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameters':

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 = 0.5029321
zero_point_input_1 = -127
scale_input_2 =0.5029321
zero_point_input_2 = -127
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette37/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=scaler.transform(x)
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
   
    predicted_label = 1 if probability > threshold else 0  
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Training and Converting Model to Tflite for Cuvette 0

In [ ]:
###FOR CUVETTE 0 ONLY####
train_df=pd.read_csv('Cuvette/Cuvette0/data_train.csv')
test_df=pd.read_csv('Cuvette/Cuvette0/data.csv')
concat=pd.concat([train_df,test_df],ignore_index=True)
concat.to_csv('jointcuvette0.csv')
joint_df=pd.read_csv('jointcuvette0.csv')
train_csv=joint_df.iloc[:,2:]
x=train_csv.iloc[:,:-1]
y=train_csv.iloc[:,-1]
scaler=StandardScaler()
x_train=scaler.fit_transform(x)

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
def create_modified_model():
    num_features = 10  
    wide_input = Input(shape=(num_features,))
    deep_input = Input(shape=(num_features,))
    hidden1 = Dense(200,activation='relu')(deep_input)
    hidden2 = Dense(190,activation='relu')(hidden1)
    hidden3 = Dense(180,activation='relu')(hidden2)
    hidden4 = Dense(170,activation='relu')(hidden3)
    hidden5 = Dense(160,activation='relu')(hidden4)
    hidden6 = Dense(150,activation='relu')(hidden5)
    hidden7 = Dense(100,activation='relu')(hidden6)
    hidden8 = Dense(80,activation='relu')(hidden7)
    hidden9 = Dense(60,activation='relu')(hidden8)
    hidden10 = Dense(32,activation='relu')(hidden9)
    parallel_input2= Dense(128,activation='relu')(deep_input)
    parallel_input3= Dense(64, activation='relu')(deep_input)
    parallel_input4= Dense(32,activation='relu')(wide_input)
    parallel_input1= Dense(256, activation='relu')(deep_input)
    parallel_input5 = Dense(512, activation='relu')(deep_input)
    concatenated_layers = concatenate([wide_input,hidden10,parallel_input1,parallel_input2,parallel_input3,parallel_input4,parallel_input5])
    
    output = Dense(1, activation='sigmoid')(concatenated_layers)
    model = Model(inputs=[wide_input, deep_input], outputs=output)
    return model


base_model = create_modified_model()
annotated_model = quantize_annotate_model(base_model)

quant_aware_model = quantize_apply(annotated_model)

quant_aware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','Recall','Precision'])
quant_aware_model.summary()


In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='model_cuvette0',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
quant_aware_model.fit((x_train,x_train),y,epochs=15,callbacks=[model_checkpoint_callback])

In [ ]:
model = tf.keras.models.load_model('model_cuvette0')

model.evaluate((x_test,x_test), y_test)

In [ ]:
def representative_dataset_gen():
    for i in range(x_train.shape[0]):
        sample = x_train[i:i+1].astype(np.float32) 
        yield [sample, sample]

In [ ]:
MODEL_DIR='model_cuvette0'
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()
with open('model0-QAT_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

In [ ]:
def quantize_input(data, scale, zero_point):
    normalized_data = (data / scale) + zero_point
    quantized_data = np.clip(normalized_data, -128, 127).astype(np.int8)
    return quantized_data
def dequantize_output(data, scale, zero_point):
    return (data.astype(np.float32) - zero_point) * scale
scale_input_1 = 0.49051696
zero_point_input_1 = -128
scale_input_2 =0.4905169
zero_point_input_2 = -128
scale_output = 0.00390625
zero_point_output = -128

In [ ]:
df=pd.read_csv('Cuvette/Cuvette0/data.csv')
df=df.iloc[:,1:]
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x=scaler.transform(x)
correct_predictions = 0
total_predictions = 0
threshold = 0.5
for i in range(len(x)):  
    input = x[i].astype(np.float32)
    input = np.expand_dims(input, axis=0)  
    quantized_input_1 = quantize_input(input, scale_input_1, zero_point_input_1)
    quantized_input_2 = quantize_input(input, scale_input_2, zero_point_input_2)
    interpreter.set_tensor(input_details[0]['index'], quantized_input_1)
    interpreter.set_tensor(input_details[1]['index'], quantized_input_2)
    
   
    interpreter.invoke()

    quantized_output = interpreter.get_tensor(output_details[0]['index'])[0]
    probability = dequantize_output(quantized_output, scale_output,zero_point_output)
   
    predicted_label = 1 if probability > threshold else 0  
    if predicted_label == y[i]:
        correct_predictions += 1
    total_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Extracting Data for Inference and Saving in .H format
## Change the Cuvette Numbers to save data of different Cuvettes. Also change the respective cuvette numbers of the file being saved

In [ ]:
file_path = 'Cuvette0/data.csv' ### Change Cuvette Number Here 
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('=', '')

measurements = df.iloc[:, 1:]  
formatted_str = "MeasurementData0 measurements0[] = {\n" ### Change Cuvette Number Here ###
for index, row in measurements.iterrows():
    formatted_str += "    {" + ", ".join(map(str, row.values)) + "},\n"
formatted_str = formatted_str.strip(',\n') + "\n};"

output_file_path = 'tflite0.h' ### Change Cuvette Number Here ###

with open(output_file_path, 'w') as file: 
    file.write(formatted_str)

print(f"Data saved to {output_file_path}")


# Extracting Mean and Std Dev Values for Scaling Data for each Cuvette

In [217]:
import os
def process_csv_file(file_number):
    file_path = f'Cuvette/Cuvette{file_number}/data_train.csv'
    
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
        return None, None
    
    df = pd.read_csv(file_path)
    df=df.iloc[:,1:]
    x=df.iloc[:,:-1]
    y=df.iloc[:,-1]

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    mean = scaler.mean_
    std_dev = scaler.scale_
    
    print(f'Cuvette{file_number}: Mean = {mean}, Std Dev = {std_dev}')
    
    return mean, std_dev
file_numbers = [0, 9 ,22, 29,37,45,47,]

for file_number in file_numbers:
    process_csv_file(file_number)

Cuvette0: Mean = [ 5877.16034771  5853.1180876   5898.2671013   6230.81216984
  6144.21220328  5313.55600134  6394.80966232  5799.38943497
  7687.98324975 17319.17853561], Std Dev = [ 34040.66973224  32555.89200022  35204.9188014   38370.11124596
  38272.84587819  37246.49561343  39360.85567054  50842.48953944
  99405.48895803 232254.43844386]
Cuvette9: Mean = [263005.00630163 107228.73537427 182464.92101678 305580.3369667
 315750.93708592 316616.04445201 321487.12511283 335841.54783275
 234127.44907605 311829.21977348], Std Dev = [ 870806.30216227  325278.70199715  553725.92898619 1083730.05335389
 1121533.38050594 1125740.11780339 1141739.68742835 1194981.44422409
  712991.48054722 1057456.74927629]
Cuvette22: Mean = [214696.88924557  23818.43831744  57672.67536614 106311.83428134
 113646.35336342 114540.78697207 119389.56726839 129504.56879257
  97153.15690565 295715.22246253], Std Dev = [ 730746.43161753   72752.5896946   176497.21949464  350258.21619293
  375907.8088742   379821.5